# Programming Language DataBase (PLDB) Visualization

In [33]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm

In [2]:
%load_ext autoreload
%autoreload 2

In [20]:
tqdm.pandas()

## Load and preprocess

In [3]:
pldb_raw = pd.read_csv("./data/pldb.csv")
pldb_raw.head()

/home/fyy/miniconda3/envs/modern_nlp/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (15,16,17,25,30,38,41,44,46,47,48,49,50,51,52,53,56,59,60,61,63,69,71,72,73,75,76,78,79,80,81,82,83,84,85,87,89,91,92,93,94,95,96,97,98,99,100,101,103,104,110,111,112,113,114,115,116,117,119,121,122,123,126,128,129,130,131,132,133,134,135,137,138,142,143,144,145,146,147,148,150,151,156,157,158,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,183,184,185,186,187,188,189,190,191,192,194,195,196,197,198,200,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310

,title,appeared,type,pldbId,rank,languageRank,factCount,lastActivity,exampleCount,bookCount,...,features.hasIds,ebook,features.hasExports,features.hasZippers,features.hasMonads,extensionOf,zulip,features.hasImplicitArguments,features.hasDynamicTyping,features.hasMethodChaining
0,Java,1995,pl,java,0,0.0,920,2022,20,401,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,JavaScript,1995,pl,javascript,1,1.0,818,2022,54,351,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,True
2,C,1972,pl,c,2,2.0,426,2022,26,78,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Python,1991,pl,python,3,3.0,691,2022,35,342,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,SQL,1974,queryLanguage,sql,4,4.0,324,2022,6,182,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
SELECTED_COLUMNS = [
    "title", "appeared", "type", "rank", "factCount", "lastActivity",
    "exampleCount", "bookCount", "paperCount", "numberOfUsers", "numberOfJobs",
    "originCommunity", "country", "wikipedia"
]

In [12]:
pldb_raw = pldb_raw[SELECTED_COLUMNS]
pldb_raw.head()

,title,appeared,type,rank,factCount,lastActivity,exampleCount,bookCount,paperCount,numberOfUsers,numberOfJobs,originCommunity,country,wikipedia
0,Java,1995,pl,0,920,2022,20,401,37,5550123,85206,Sun Microsystems,United States,https://en.wikipedia.org/wiki/Java_(programmin...
1,JavaScript,1995,pl,1,818,2022,54,351,48,5962666,63993,Netscape,United States,https://en.wikipedia.org/wiki/JavaScript
2,C,1972,pl,2,426,2022,26,78,19,3793768,59919,Bell Labs,United States,https://en.wikipedia.org/wiki/C_(programming_l...
3,Python,1991,pl,3,691,2022,35,342,52,2818037,46976,Centrum Wiskunde & Informatica,Netherlands,https://en.wikipedia.org/wiki/Python_(programm...
4,SQL,1974,queryLanguage,4,324,2022,6,182,37,7179119,219617,IBM,United States,https://en.wikipedia.org/wiki/SQL


In [13]:
print(
    "Percentage of NaN values in the PLDB dataset (total {} rows):".format(len(pldb_raw))
)
print((pldb_raw.isna().sum() / len(pldb_raw) * 100).sort_values()[:20])

Percentage of NaN values in the PLDB dataset (total 4818 rows):
title               0.000000
appeared            0.000000
type                0.000000
rank                0.000000
factCount           0.000000
lastActivity        0.000000
exampleCount        0.000000
bookCount           0.000000
paperCount          0.000000
numberOfUsers       0.000000
numberOfJobs        0.000000
originCommunity    27.791615
country            32.004981
wikipedia          61.623080
dtype: float64


In [14]:
pldb = pldb_raw.fillna({
    "originCommunity": "Unknown",
    "country": "Unknown"
})

In [44]:
def extract_text(url):
    try:
        response = requests.get(url, timeout=10) # set timeout to 10 seconds
        soup = BeautifulSoup(response.text, "html.parser")
        first_section = soup.find("h2")
        first_paragraph = soup.find("p")
        if first_section is None or first_paragraph is None:
            return None
        if first_section.start() < first_paragraph.start():
            return None
        return soup.text[first_paragraph.start():first_section.start()].strip()
    except (requests.exceptions.Timeout, AttributeError):
        return None

In [45]:
pldb.loc[pldb["wikipedia"].notna(), "wikipedia"] = pldb.loc[pldb["wikipedia"].notna(), "wikipedia"].progress_apply(extract_text)

 10%|█         | 1/10 [00:00<00:03,  2.97it/s]


TypeError: 'NoneType' object is not callable